### Preprocess the data. Balance the dataset. Create 3 datasets: training, validation and test. Save the sets in .npz format

### Extract the data from the csv file

In [1]:
import numpy as np
from sklearn import preprocessing

# Numpy library has functions used for working with arrays and matrices
# faster, clearer and better quality code using Numpy

# Tensorflow is an open-source Machine Learning library, particular focus on training and inference of deep neural networks

# TensorFlow Datasets is a collection of datasets ready to use with TensorFlow

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

# importing the dataset

In [ ]:
unscaled_inputs_all = raw_csv_data[:,1:-1]

# this takes all the data in the csv excep the 1st and last column
# 1st column is arbitrarily chosen ID and last column is the target

targets_all = raw_csv_data[:,-1]
# this variable stores just the targets (in the last column)

### Balance the Dataset

In [2]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

# counts how many targets are 1s
# keep as many 0s as there are 1s (and delete the rest)
# the last variable removes the indices (using a tuple)

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter +=1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
# the shape of 'targets_all' on axis = 0, basically the length of the vector
# increase the counter by 1 if the target is 0
# when the count of 0s matches the number of 1s, then the rest can be removed

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

# np.delete is a method that deletes an object along an axis
# now the dataset will be balanced with an equal number of 1s and 0s
# and stored in the variable 'targets_equal_priors'

### Standardizing the inputs

In [3]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# preprocessing.scale is the library imported from sklearn to standardize an array along an axis

### Shuffle the data

In [4]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# as the data is being batched, we must shuffle the inputs and the targets (keeps the same info but in randomized order)
# np.arange([start], stop) is a method that returns evenly spaced values within a given interval
# the np.random.shuffle method is used to shuffle the numbers in a sequence

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]
# new variables to store the shuffled, scaled inputs and another to store the shuffled, correct number of targets

### Split the dataset into training, validation and test

In [5]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

# using the 80% (train), 10% (validate), 10% (test) split for the datasets

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# applying the shuffling the training inputs and the targets

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

# the validation inputs will be the interval from train_samples_count + validation_samples_count
# same with the validation_targets

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

# the test inputs and targets are from everything that's left

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(train_targets / train_samples_count))
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

# we want our priors to all be around 50% (between 45-55) so the 3 sets are balanced - which is what we get here

1787.0 3579 0.4993014808605756
222.0 447 0.4993014808605756
228.0 448 0.5089285714285714


### Save the 3 datasets using the .npz method

In [6]:
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

### All this code above can be used to process ANY dataset with 2 classes